In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
root_path = '/content/drive/MyDrive/ColabNotebooks/zindi/GenderBaseViolence/'

## Gender Based Violence - Tweet classification challenge 

This challenge aims to classify tweets about GBV without using keywords.

Are you able to develop a machine learning model that can classify a tweet about gender-based violence into either of the five categories:
- Sexual violence
- Emotional violence
- Economic violence
- Physical violence
- Harmful traditional practice


### Install modules/package

In [3]:
!pip install -U nltk #natural language toolkit

     |████████████████████████████████| 1.5 MB 9.1 MB/s 
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


### import libraries/modules

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns


from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

### load the dataset

In [5]:
train = pd.read_csv(root_path + "Train.csv")

In [6]:
train.head()

,Tweet_ID,tweet,type
0,ID_0022DWKP,Had a dream i got raped last night. By a guy i...,sexual_violence
1,ID_00395QYM,he thought the word raped means sex and told m...,sexual_violence
2,ID_003EOSSF,She NOT TALKING TO ME I WAS RAPED BY 2 MEN 1 M...,sexual_violence
3,ID_004BBHOD,I was sexually abused for 3 years at age 4 to ...,sexual_violence
4,ID_004F7516,Chessy Prout can do better by telling the trut...,sexual_violence


In [7]:
test = pd.read_csv(root_path + "Test.csv")

In [8]:
test.head()

,Tweet_ID,tweet
0,ID_0095QL4S,"because he was my boyfriend, and if I said no,..."
1,ID_00DREW5O,"lol no, I'm telling you it's not legal. It's l..."
2,ID_00E9F5X9,Somalia's semi-autonomous Puntland region has ...
3,ID_00G9OSKZ,University of Cape Coast students being robbed...
4,ID_00HU96U6,"""Somebody came up behind him and stabbed him i..."


In [33]:
X = train.drop(["type"], axis=1)
y = train["type"]

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

CountVectorizer -converts text files/data into numerical feature vectors. You can find more information on CountVectorizer  here  https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [15]:
vectorizer = CountVectorizer()

# fit_transform our tweets values to numerical vectors
x_train_counts = vectorizer.fit_transform(X_train["tweet"])
x_train_counts

<26565x30352 sparse matrix of type '<class 'numpy.int64'>'
	with 824624 stored elements in Compressed Sparse Row format>

In [16]:
print(vectorizer.get_feature_names())

['00', '000', '00006', '000th', '0093', '00am', '01', '010413', '01am', '02', '026', '03', '04', '05', '06', '07780', '08', '08146229938', '09', '09091177639', '0this', '10', '100', '1000', '10000', '1000000', '1000000000', '1000s', '1000th', '1000x', '100cedis', '100k', '100l', '100m', '100millions', '100s', '100thieves', '100x', '101', '1012', '102', '10221582408610984', '103', '103730280384th', '104', '105', '108', '10am', '10f', '10k', '10million', '10mins', '10mo', '10month', '10pm', '10s', '10th', '10times', '10x', '10year', '10years', '10yo', '10yrs', '11', '110', '1100', '110k', '111', '1130pm', '1152', '11f', '11i', '11k', '11pm', '11th', '11year', '11yo', '11yr', '11yrs', '12', '120', '1200', '1207', '120lbs', '12137', '122', '123', '1236', '1240', '1247', '125', '1260', '127', '1271', '1274', '1275936441', '1276', '1285', '1293', '1296', '1299', '12a', '12hrs', '12mth', '12pm', '12th', '12y', '12year', '12yo', '12yr', '12yrs', '13', '130', '130k', '1311', '1312', '132', '13f

Using the CountVectorizer is effective, but this might give more preference to longer tweets than others. Let's try using frequencies. https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html

In [17]:
tfidf_transformer = TfidfTransformer()
x_train_tfidf = tfidf_transformer.fit_transform(x_train_counts)
x_train_tfidf.shape

(26565, 30352)

Multiple algorithms are used for text classification; let’s look at:-

### 1. Naive Bayes MultinomialNB https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

In [18]:
# train.type.values our y values
clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                  ('clf', MultinomialNB())])
clf = clf.fit(X_train["tweet"], y_train)

### evaluation metric
The evaluation metric on the Zindi leaderboard is accuracy; let’s test our simple model,

In [19]:
predicted = clf.predict(X_test.tweet)

In [20]:
predicted

array(['sexual_violence', 'sexual_violence', 'sexual_violence', ...,
       'sexual_violence', 'sexual_violence', 'sexual_violence'],
      dtype='<U28')

In [21]:
np.unique(predicted)

array(['Physical_violence', 'emotional_violence', 'sexual_violence'],
      dtype='<U28')

In [22]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,predicted)

0.8871226595338173

- Our simple model gave us an accuracy score of 88%, fair enough, but you should also be careful that the data is highly imbalanced, as discussed earlier. 

- Question? Is the imbalance percentage the same as in the test? Can you think of various ways to deal with the imbalance dataset

- We can also see the only 3 categories(i.e 'Physical_violence', 'emotional_violence', 'sexual_violence') were predicted yet we have 5 categories. Therefore 88% might not be that good.


### submission

In [25]:
sample_submission = pd.read_csv(root_path + "SampleSubmission.csv")

In [26]:
sample_submission.head()

,Tweet_ID,type
0,ID_0095QL4S,NaN
1,ID_00DREW5O,NaN
2,ID_00E9F5X9,NaN
3,ID_00G9OSKZ,NaN
4,ID_00HU96U6,NaN


In [27]:
##let's prdict on the test data
test_predict = clf.predict(test.tweet)

In [28]:
test_predict

array(['sexual_violence', 'sexual_violence', 'sexual_violence', ...,
       'sexual_violence', 'sexual_violence', 'sexual_violence'],
      dtype='<U28')

In [29]:
np.unique(test_predict)

array(['Physical_violence', 'sexual_violence'], dtype='<U28')

In [30]:
sample_submission["type"] = test_predict

In [32]:
sample_submission.to_csv(root_path + 'baseline_sub.csv', index=False)

### To do,
- Do more analysis
- Try working on ways to balance the dataset, undersampling, oversampling, using SMOTE, etc
- You can try other text classification models, e.g., using nltk, etc.
